<a href="https://www.kaggle.com/code/sagnikdebnath007/fusion-model-autoencodings?scriptVersionId=204841255" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import pickle
with open('/kaggle/input/meld-embeddings/text_embeddings.pkl', 'rb') as f:
    text_emb = pickle.load(f)

In [2]:
text_emb[0].shape

(98304,)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [4]:
device1=torch.device('cuda:0')
device2=torch.device('cuda:1')

In [5]:
text_emb=torch.tensor(text_emb,dtype=torch.float32).to(device2)

/tmp/ipykernel_30/2251793622.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  text_emb=torch.tensor(text_emb,dtype=torch.float32).to(device2)


In [6]:
text_emb.shape

torch.Size([9989, 98304])

In [7]:
input_dim=98304
output_dim=1611

In [8]:
class Autoencoder(nn.Module):
    def __init__(self,input_dim,output_dim,dropout=0.5):
        super(Autoencoder,self).__init__()
        self.enc_fc1=nn.Linear(input_dim,2048)
        self.enc_fc2=nn.Linear(2048,1024)
        self.enc_fc3=nn.Linear(1024,output_dim)
        self.relu=nn.ReLU()
        self.dropout=nn.Dropout(dropout)
        self.dec_fc1=nn.Linear(output_dim,1024)
        self.dec_fc2=nn.Linear(1024,2048)
        self.dec_fc3=nn.Linear(2048,input_dim)
        
        self.encoder=nn.Sequential(
            self.enc_fc1,
            self.relu,
            self.enc_fc2,
            self.relu,
            self.dropout,
            self.enc_fc3
        )
        self.decoder=nn.Sequential(
            self.dec_fc1,
            self.relu,
            self.dec_fc2,
            self.relu,
            self.dropout,
            self.dec_fc3
        )
    def forward(self,x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

        

In [9]:
model = Autoencoder(input_dim,output_dim).to(device2)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

In [10]:
dataset = TensorDataset(text_emb,text_emb)  
data_loader = DataLoader(dataset, batch_size=8, shuffle=True)

In [11]:
torch.cuda.empty_cache()

In [12]:
epochs=10
from tqdm import tqdm
for epoch in range(epochs):
    running_loss=0.0
    for batch_data, _ in tqdm(data_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        batch_data = batch_data.to(device2) 
        optimizer.zero_grad()
        reconstructed = model(batch_data)
        loss = criterion(reconstructed, batch_data)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
    avg_loss = running_loss / len(data_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")
    torch.cuda.empty_cache()

Epoch 1/10: 100%|██████████| 1249/1249 [03:02<00:00,  6.85it/s]


Epoch [1/10], Loss: 0.0772


Epoch 2/10: 100%|██████████| 1249/1249 [03:01<00:00,  6.87it/s]


Epoch [2/10], Loss: 0.0646


Epoch 3/10: 100%|██████████| 1249/1249 [03:01<00:00,  6.87it/s]


...

Epoch [8/10], Loss: 0.0591


Epoch 9/10: 100%|██████████| 1249/1249 [03:01<00:00,  6.87it/s]


Epoch [9/10], Loss: 0.0589


Epoch 10/10: 100%|██████████| 1249/1249 [03:01<00:00,  6.87it/s]

Epoch [10/10], Loss: 0.0587


In [13]:
with torch.no_grad():
    text_encodings = model.encoder(text_emb).cpu()

In [14]:
text_encodings.shape

torch.Size([9989, 1611])

In [15]:
bert_encodings_np = [encoding.cpu().numpy() for encoding in text_encodings]

In [16]:
bert_encodings_np[0].shape

(1611,)

In [17]:
with open('text_encodings_reduced.pkl', 'wb') as f:
    pickle.dump(bert_encodings_np, f)

In [18]:
with open('text_encodings_reduced.pkl', 'rb') as f:
    loaded_array_list = pickle.load(f)
len(loaded_array_list[0])

1611